In [ ]:
import sys
import os
from IPython.display import clear_output

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D
from tensorflow.keras import Model

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.98
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))     
tf.compat.v1.enable_eager_execution()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append("./lib/")
from models_varia_mse_y import *
from BDC_utils import *
from SNM_GRUD import *
from SNM_Interpolate import *
from SNM_SupNotMIWAE import *

batch = 500
model_path = './model/'

include_init25s = True
# 처음 25초를 outcome에 포함시켜서 input과 같은 길이를 얻으려면 True
# 처음 25초를 outcome에서 제외하려면 False

# Loading input data

In [ ]:
# downstream task의 input data
file_path = './inputdata/'
filename = 'abp_input_sample.npy'
abp_input = np.load(file_path+filename) # (length, time)의 shape 가정

# HIVAE

### Loading model

In [ ]:
encoder = JointEncoderGRU
decoder = GaussianDecoder

HIVAEModel = HI_VAE(latent_dim=10, data_dim=50, time_length=60,
                    encoder_sizes=[100, 80, 60], encoder=encoder,
                    decoder_sizes=[60,80,100], decoder=decoder,
                    M=1, K=1, beta=0.1)
_ = tf.compat.v1.train.get_or_create_global_step()
trainable_vars = HIVAEModel.get_trainable_vars()
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3)

HIVAEModel.encoder.net.load_weights(model_path+'HIVAE_encoder.hdf5')
HIVAEModel.decoder.net.load_weights(model_path+'HIVAE_decoder.hdf5')

### Generating cleansed data

In [ ]:
# frame으로 분할 - (length, time, data_dim=50)의 shape
abp_fr = tf.signal.frame(abp_input.astype('float32'), 50, 50).numpy()

# batch 단위로 prediction
for n in range(abp_fr.shape[0]//batch):
    
    # 0~30초를 입력으로 0~30초 outcome (참고로 frame은 0.5초 단위)
    x = abp_fr[batch*n:batch*(n+1),:60,:]
    outcome = HIVAEModel.decode(HIVAEModel.encode(x).mean().numpy()).mean().numpy()
    
    # ABP input의 25~30초를 outcome의 25~30초로 대체
    x = np.concatenate((x[:,:50,:], outcome[:,-10:,:]), axis=1)
    
    if include_init25s == True:
        # ABP input의 처음 25초 포함
        outsig_entire_time = outcome.copy()
    else:
        # ABP input의 처음 25초 제외
        outsig_entire_time = outcome[:,-10:,:].copy()

    # Sliding prediction, ABP input 5초씩 새로 결합하여 prediction
    for i in range(1,(abp_fr.shape[1]-50)//10):
        
        # x의 후반 25초에 ABP input 5초를 새로 결합
        x_h = np.concatenate((x[:,10:,:], abp_fr[batch*n:batch*(n+1),60+10*(i-1):60+10*i,:]), axis=1)
        
        # 만약 nan이 있다면 0으로 대체
        if np.sum(np.isnan(x_h)) > 0:
            x_h = np.where(np.isnan(x_h), 0.0, x_h)
        
        # x_h를 입력으로 outcome
        outcome = HIVAEModel.decode(HIVAEModel.encode(x_h).mean().numpy()).mean().numpy()
        
        # x_h의 후반 5초를 outcome의 후반 5초로 대체
        x = np.concatenate((x_h[:,:50,:], outcome[:,-10:,:]), axis=1)
        
        # 새로운 outcome의 후반 5초를 계속해서 결합
        outsig_entire_time = np.concatenate((outsig_entire_time, outcome[:,-10:,:]), axis=1)
        
    # batch 단위로 reshape 하여 결합
    if n == 0:
        outsig_entire_data_HIVAE = np.reshape(outsig_entire_time, (outsig_entire_time.shape[0],-1))
    else:
        outsig_entire_data_HIVAE = np.concatenate((outsig_entire_data_HIVAE, 
                                                   np.reshape(outsig_entire_time, (outsig_entire_time.shape[0],-1))), 
                                                  axis=0)
        
    print(f'{outsig_entire_data_HIVAE.shape[0]} / {abp_input.shape[0]}')
    clear_output(wait=True)
print(f'{outsig_entire_data_HIVAE.shape[0]} / {abp_input.shape[0]} -- HIVAE prediction complete')

# GPVAE

### Loading model

In [ ]:
encoder = BandedJointEncoderGRU
decoder = GaussianDecoder

GPVAEModel = GP_VAE(latent_dim=10, data_dim=50, time_length=60,
                    encoder_sizes=[100, 80, 60], encoder=encoder,
                    decoder_sizes=[60,80,100], decoder=decoder,
                    M=1, K=1, beta=0.1)
_ = tf.compat.v1.train.get_or_create_global_step()
trainable_vars = GPVAEModel.get_trainable_vars()
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3)

GPVAEModel.encoder.net.load_weights(model_path+'GPVAE_encoder.hdf5')
GPVAEModel.decoder.net.load_weights(model_path+'GPVAE_decoder.hdf5')

### Generating cleansed data

In [ ]:
# frame으로 분할 - (length, time, data_dim=50)의 shape
abp_fr = tf.signal.frame(abp_input.astype('float32'), 50, 50).numpy()

# batch 단위로 prediction
for n in range(abp_fr.shape[0]//batch):
    
    # 0~30초를 입력으로 0~30초 outcome (참고로 frame은 0.5초 단위)
    x = abp_fr[batch*n:batch*(n+1),:60,:]
    outcome = GPVAEModel.decode(GPVAEModel.encode(x).mean().numpy()).mean().numpy()
    
    # ABP input의 25~30초를 outcome의 25~30초로 대체
    x = np.concatenate((x[:,:50,:], outcome[:,-10:,:]), axis=1)
    
    if include_init25s == True:
        # ABP input의 처음 25초 포함
        outsig_entire_time = outcome.copy()
    else:
        # ABP input의 처음 25초 제외
        outsig_entire_time = outcome[:,-10:,:].copy()

    # Sliding prediction, ABP input 5초씩 새로 결합하여 prediction
    for i in range(1,(abp_fr.shape[1]-50)//10):
        
        # x의 후반 25초에 ABP input 5초를 새로 결합
        x_h = np.concatenate((x[:,10:,:], abp_fr[batch*n:batch*(n+1),60+10*(i-1):60+10*i,:]), axis=1)
        
        # 만약 nan이 있다면 0으로 대체
        if np.sum(np.isnan(x_h)) > 0:
            x_h = np.where(np.isnan(x_h), 0.0, x_h)
        
        # x_h를 입력으로 outcome
        outcome = GPVAEModel.decode(GPVAEModel.encode(x_h).mean().numpy()).mean().numpy()
        
        # x_h의 후반 5초를 outcome의 후반 5초로 대체
        x = np.concatenate((x_h[:,:50,:], outcome[:,-10:,:]), axis=1)
        
        # 새로운 outcome의 후반 5초를 계속해서 결합
        outsig_entire_time = np.concatenate((outsig_entire_time, outcome[:,-10:,:]), axis=1)
        
    # batch 단위로 reshape 하여 결합
    if n == 0:
        outsig_entire_data_GPVAE = np.reshape(outsig_entire_time, (outsig_entire_time.shape[0],-1))
    else:
        outsig_entire_data_GPVAE = np.concatenate((outsig_entire_data_GPVAE, 
                                                   np.reshape(outsig_entire_time, (outsig_entire_time.shape[0],-1))), 
                                                  axis=0)
        
    print(f'{outsig_entire_data_GPVAE.shape[0]} / {abp_input.shape[0]}')
    clear_output(wait=True)
print(f'{outsig_entire_data_GPVAE.shape[0]} / {abp_input.shape[0]} -- GPVAE prediction complete')

# BDC

### Loading model

In [ ]:
Length = 3000
OrigDim = 1
NumLayers = 2
EmbedDim = 64
NumHead = 1
DimFC = 200

InpLayer = Input((Length, 1))
InpMask  = Input((Length, 1))

EmbedOut = BDCEncoder(num_layers = NumLayers, d_model = EmbedDim, num_heads = NumHead, dff = DimFC, t_len=Length)(InpLayer, True, InpMask)

Projection = Conv1D(filters=OrigDim, kernel_size=11, strides=1, padding='same', dilation_rate=1)(EmbedOut)

BDCModel = Model([InpLayer,InpMask ], Projection)
BDCModel.compile(loss='mse', optimizer = 'adam')
BDCModel.load_weights(model_path+'BDC.hdf5')

### Generating cleansed data

In [ ]:
# input shape 변경 - (length, time, data_dim=1)의 shape
abp_resh = np.reshape(abp_input, (abp_input.shape[0],-1,1))

# 초기 25초만 observed(1), 이후는 missing(0) 가정
mask = np.zeros_like(abp_resh)
mask[:,:2500,:] = np.ones_like(abp_resh[:,:2500,:])

# batch 단위로 prediction
for n in range(abp_resh.shape[0]//batch):
    
    # 0~30초의 ABP와 mask를 입력으로 0~30초 ABP outcome
    outcome = BDCModel.predict([abp_resh[batch*n:batch*(n+1),:3000,:], mask[batch*n:batch*(n+1),:3000,:]])
    
    # ABP input의 25~30초를 outcome의 25~30초로 대체
    x = np.concatenate((abp_resh[batch*n:batch*(n+1),:2500,:], outcome[:,-500:,:]), axis=1)
    
    # outcome으로 대체된 시간대의 mask는 observed(1)로 변경
    m = np.concatenate((mask[batch*n:batch*(n+1),:2500,:], np.ones_like(outcome[:,-500:,:])), axis=1)
    
    if include_init25s == True:
        # ABP input의 처음 25초 포함
        outsig_entire_time = outcome.copy()
    else:
        # ABP input의 처음 25초 제외
        outsig_entire_time = outcome[:,-500:,:].copy()
    
    # Sliding prediction, ABP input 5초씩 새로 결합하여 prediction
    for i in range(1,(abp_resh.shape[1]-2500)//500):
        
        # x 후반 25초에 ABP input 5초를 새로 결합, mask도 같은 방식으로 결합
        x_h = np.concatenate((x[:,500:,:],abp_resh[batch*n:batch*(n+1),3000+500*(i-1):3000+500*i,:]), axis=1)
        m_h = np.concatenate((m[:,500:,:],mask[batch*n:batch*(n+1),3000+500*(i-1):3000+500*i,:]), axis=1)
        
        # 만약 nan이 있다면 0으로 대체
        if np.sum(np.isnan(x_h)) > 0:
            x_h = np.where(np.isnan(x_h), 0.0, x_h)
        
        # x_h와 m_h를 입력으로 ABP outcome
        outcome = BDCModel.predict([x_h, m_h])
        
        # x_h의 후반 5초를 outcome의 후반 5초로 대체
        x = np.concatenate((x_h[:,:2500,:], outcome[:,-500:,:]), axis=1)
        
        # outcome으로 대체된 시간대의 mask는 observed(1)로 변경
        m = np.concatenate((m_h[:,:2500,:], np.ones_like(outcome[:,-500:,:])), axis=1)
        
        # 새로운 outcome의 후반 5초를 계속해서 결합
        outsig_entire_time = np.concatenate((outsig_entire_time, outcome[:,-500:,:]), axis=1)
    
    # batch 단위로 reshape 하여 결합
    if n == 0:
        outsig_entire_data_BDC = np.reshape(outsig_entire_time, (outsig_entire_time.shape[0],-1))
    else:
        outsig_entire_data_BDC = np.concatenate((outsig_entire_data_BDC, 
                                                 np.reshape(outsig_entire_time, (outsig_entire_time.shape[0],-1))), 
                                                axis=0)

    
    print(f'{outsig_entire_data_BDC.shape[0]} / {abp_input.shape[0]}')
    clear_output(wait=True)
print(f'{outsig_entire_data_BDC.shape[0]} / {abp_input.shape[0]} -- BDC prediction complete')

# SupNotMIWAE

### Loading model

In [ ]:
Length = 60
OrigDim = 50
InpLayer = tf.keras.layers.Input((Length, OrigDim))
InpMask  = tf.keras.layers.Input((Length, OrigDim))
ImputeOut = SupNotMIWAE(n_train_latents=10, n_train_samples=1)([InpLayer, InpMask])

SNMModel = Model([InpLayer,InpMask ], ImputeOut)
SNMModel.compile(loss='mse', optimizer='adam')
SNMModel.load_weights(model_path+'SupNotMIWAE.hdf5')

### Generating cleansed data

In [ ]:
# input shape 변경 - (length, time, data_dim=50)의 shape
abp_fr = tf.signal.frame(abp_input.astype('float32'), 50, 50).numpy()

# 초기 25초만 observed(1), 이후는 missing(0) 가정
mask = np.zeros_like(abp_fr)
mask[:,:50,:] = np.ones_like(abp_fr[:,:50,:])

# batch 단위로 prediction
for n in range(abp_fr.shape[0]//batch):
    
    # 0~30초의 ABP와 mask를 입력으로 0~30초 ABP outcome
    outcome = SNMModel.predict([abp_fr[batch*n:batch*(n+1),:60,:], mask[batch*n:batch*(n+1),:60,:]])
    
    # ABP input의 25~30초를 outcome의 25~30초로 대체
    x = np.concatenate((abp_fr[batch*n:batch*(n+1),:50,:], outcome[:,-10:,:]), axis=1)
    
    # outcome으로 대체된 시간대의 mask는 observed(1)로 변경
    m = np.concatenate((mask[batch*n:batch*(n+1),:50,:], np.ones_like(outcome[:,-10:,:])), axis=1)
    
    if include_init25s == True:
        # ABP input의 처음 25초 포함
        outsig_entire_time = outcome.copy()
    else:
        # ABP input의 처음 25초 제외
        outsig_entire_time = outcome[:,-10:,:].copy()
    
    # Sliding prediction, ABP input 5초씩 새로 결합하여 prediction
    for i in range(1,(abp_fr.shape[1]-50)//10):
        
        # x 후반 25초에 ABP input 5초를 새로 결합, mask도 같은 방식으로 결합
        x_h = np.concatenate((x[:,10:,:],abp_fr[batch*n:batch*(n+1),60+10*(i-1):60+10*i,:]), axis=1)
        m_h = np.concatenate((m[:,10:,:],mask[batch*n:batch*(n+1),60+10*(i-1):60+10*i,:]), axis=1)
        
        # 만약 nan이 있다면 0으로 대체
        if np.sum(np.isnan(x_h)) > 0:
            x_h = np.where(np.isnan(x_h), 0.0, x_h)
        
        # x_h와 m_h를 입력으로 ABP outcome
        outcome = SNMModel.predict([x_h, m_h])
        
        # x_h의 후반 5초를 outcome의 후반 5초로 대체
        x = np.concatenate((x_h[:,:50,:], outcome[:,-10:,:]), axis=1)
        
        # outcome으로 대체된 시간대의 mask는 observed(1)로 변경
        m = np.concatenate((m_h[:,:50,:], np.ones_like(outcome[:,-10:,:])), axis=1)
        
        # 새로운 outcome의 후반 5초를 계속해서 결합
        outsig_entire_time = np.concatenate((outsig_entire_time, outcome[:,-10:,:]), axis=1)
    
    # batch 단위로 reshape 하여 결합
    if n == 0:
        outsig_entire_data_SNM = np.reshape(outsig_entire_time, (outsig_entire_time.shape[0],-1))
    else:
        outsig_entire_data_SNM = np.concatenate((outsig_entire_data_SNM, 
                                                 np.reshape(outsig_entire_time, (outsig_entire_time.shape[0],-1))), 
                                                axis=0)

    
    print(f'{outsig_entire_data_SNM.shape[0]} / {abp_input.shape[0]}')
    clear_output(wait=True)
print(f'{outsig_entire_data_SNM.shape[0]} / {abp_input.shape[0]} -- SNM prediction complete')